In [1]:
!pip install datasets sympy wandb
!pip install --no-cache-dir bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 73.7 MB/s eta 0:00:00


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen1.5-1.8B-Chat"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [4]:
import json

# Đọc file gốc
with open("training_data1.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Chuyển đổi sang định dạng QA
qa_samples = []
for idx, item in enumerate(data, 1):
    conversations = item["conversations"]
    # Tìm user và assistant message
    user_msg = None
    assistant_msg = None

    for conv in conversations:
        if conv["role"] == "user":
            user_msg = conv["content"]
        elif conv["role"] == "assistant":
            assistant_msg = conv["content"]

    # Tạo sample theo format mới với Question X: và Answer X:
    if user_msg and assistant_msg:
        qa_samples.append({
            "prompt": f"Question {idx}: {user_msg}",
            "completion": f"Answer {idx}: {assistant_msg}"
        })

# Ghi ra file JSONL
with open("military_training_qa.jsonl", "w", encoding="utf-8") as f:
    for sample in qa_samples:
        json_line = json.dumps(sample, ensure_ascii=False)
        f.write(json_line + "\n")

print(f"✅ Đã chuyển đổi thành công {len(qa_samples)} câu hỏi-trả lời")
print(f"📄 File output: military_training_qa.jsonl")

# In một vài ví dụ
print("\n📋 Ví dụ 3 mẫu đầu tiên:")
for i, sample in enumerate(qa_samples[:3], 1):
    print(f"\n--- Mẫu {i} ---")
    print(f"Prompt: {sample['prompt'][:80]}...")
    print(f"Completion: {sample['completion'][:80]}...")

✅ Đã chuyển đổi thành công 37 câu hỏi-trả lời
📄 File output: military_training_qa.jsonl

📋 Ví dụ 3 mẫu đầu tiên:

--- Mẫu 1 ---
Prompt: Question 1: Chủ đề lãnh đạo năm 2025 mà Đảng ủy Sư đoàn xác định là gì?...
Completion: Answer 1: Chủ đề lãnh đạo năm 2025 là: 'Dân chủ, kỷ cương - Sẵn sàng chiến đấu c...

--- Mẫu 2 ---
Prompt: Question 2: Phong trào thi đua quyết thắng năm 2025 có chủ đề, tư tưởng chỉ đạo,...
Completion: Answer 2: Phong trào thi đua quyết thắng năm 2025: Chủ đề: 'Dân chủ, đoàn kết, k...

--- Mẫu 3 ---
Prompt: Question 3: 6 chỉ tiêu chính của phong trào thi đua quyết thắng năm 2025 là gì?...
Completion: Answer 3: Sáu chỉ tiêu: (1) Thường xuyên quán triệt sâu sắc Nghị quyết của cấp ủ...


In [5]:
!wandb login

wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: phongnguyenvan1561996 (phongnguyenvan1561996-hth-tr-ng-ph-t) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
import wandb
wandb.init(
    project="demo-yt-video",
    config={
        "learning_rate": 5e-5,
        "architecture": "DeepSeek-R1-Distill-Qwen-1.5B",
        "dataset": "dataset.jsonl",
        "epochs": 2,
    }
)

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: phongnguyenvan1561996 (phongnguyenvan1561996-hth-tr-ng-ph-t) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from datasets import load_dataset

dataset = load_dataset("json", data_files={"train": "training_data1.json"}, split="train")
print({len(dataset)})

train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

Generating train split: 0 examples [00:00, ? examples/s]

{37}


In [10]:
import json

# Đọc file gốc
with open("training_data1.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Chuyển đổi sang định dạng prompt-completion
qa_samples = []
for idx, item in enumerate(data, 1):
    conversations = item["conversations"]
    user_msg = None
    assistant_msg = None

    for conv in conversations:
        if conv["role"] == "user":
            user_msg = conv["content"]
        elif conv["role"] == "assistant":
            assistant_msg = conv["content"]

    if user_msg and assistant_msg:
        qa_samples.append({
            "prompt": f"Question {idx}: {user_msg}",
            "completion": f"Answer {idx}: {assistant_msg}"
        })

# Ghi ra file JSONL
with open("military_training_qa.jsonl", "w", encoding="utf-8") as f:
    for sample in qa_samples:
        json_line = json.dumps(sample, ensure_ascii=False)
        f.write(json_line + "\n")

print(f"✅ Đã tạo {len(qa_samples)} samples trong file military_training_qa.jsonl")

✅ Đã tạo 37 samples trong file military_training_qa.jsonl


In [11]:
from datasets import load_dataset

# Load dataset từ file JSONL đã chuyển đổi
dataset = load_dataset("json", data_files={"train": "military_training_qa.jsonl"}, split="train")

# Kiểm tra cấu trúc
print(dataset)
print("\nMẫu đầu tiên:")
print(dataset[0])

# Chia train/test
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

print(f"\nTrain: {len(train_dataset)} samples")
print(f"Eval: {len(eval_dataset)} samples")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 37
})

Mẫu đầu tiên:
{'prompt': 'Question 1: Chủ đề lãnh đạo năm 2025 mà Đảng ủy Sư đoàn xác định là gì?', 'completion': "Answer 1: Chủ đề lãnh đạo năm 2025 là: 'Dân chủ, kỷ cương - Sẵn sàng chiến đấu cao - An toàn mọi mặt'"}

Train: 29 samples
Eval: 8 samples


In [12]:
def tokenize_function(examples):
    combined_texts = [
        f"{prompt}\n{completion}"
        for prompt, completion in zip(examples["prompt"], examples["completion"])
    ]
    tokenized = tokenizer(
        combined_texts,
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

print("✅ Tokenization hoàn tất!")

Map:   0%|          | 0/29 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

✅ Tokenization hoàn tất!


In [13]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")

In [14]:
from peft import get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=8, lora_alpha=16, lora_dropout=0.05, task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,089,536 || all params: 1,778,177,536 || trainable%: 0.0613


In [16]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./deepseek_finetuned",
    num_train_epochs=5,  # Giảm từ 50 → 5 (tránh overfit với 38 samples)
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    fp16=True,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",  # ✅ Đã sửa
    eval_steps=10,
    learning_rate=3e-5,
    logging_dir="./logs",
    report_to="wandb",
    run_name="DeepSeek_Military_Training",
    load_best_model_at_end=True,  # Thêm: Load model tốt nhất
    metric_for_best_model="eval_loss",  # Thêm: Metric để đánh giá
    save_total_limit=2,  # Thêm: Chỉ giữ 2 checkpoint tốt nhất
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=5, training_loss=13.536904907226562, metrics={'train_runtime': 84.9941, 'train_samples_per_second': 1.706, 'train_steps_per_second': 0.059, 'total_flos': 688117422489600.0, 'train_loss': 13.536904907226562, 'epoch': 5.0})

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

In [18]:
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=5, training_loss=13.655445861816407, metrics={'train_runtime': 69.1932, 'train_samples_per_second': 2.096, 'train_steps_per_second': 0.072, 'total_flos': 688117422489600.0, 'train_loss': 13.655445861816407, 'epoch': 5.0})

In [19]:
save_path = "/content/drive/My Drive/deepseek_finetuned"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print({save_path})

{'/content/drive/My Drive/deepseek_finetuned'}


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Định nghĩa lại các biến cần thiết
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
save_path = "/content/drive/My Drive/deepseek_finetuned"  # Đường dẫn LoRA adapters

# Load base model (không cần quantization khi merge)
print("⏳ Đang load base model...")
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load LoRA adapters và merge vào base model
print("⏳ Đang merge LoRA adapters...")
model = PeftModel.from_pretrained(base_model, save_path)
model = model.merge_and_unload()  # Gộp LoRA vào base model

# Lưu model đã merge (model hoàn chỉnh)
final_save_path = "/content/drive/My Drive/deepseek_finetuned_full"
print(f"⏳ Đang lưu model vào {final_save_path}...")
model.save_pretrained(final_save_path)
tokenizer.save_pretrained(final_save_path)

print(f"✅ Hoàn tất! Model đã được lưu tại: {final_save_path}")

⏳ Đang load base model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


⏳ Đang merge LoRA adapters...
⏳ Đang lưu model vào /content/drive/My Drive/deepseek_finetuned_full...
✅ Hoàn tất! Model đã được lưu tại: /content/drive/My Drive/deepseek_finetuned_full


In [1]:
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, save_path)
model = model.merge_and_unload()

final_save_path = "/content/drive/My Drive/deepseek_finetuned_full"
model.save_pretrained(final_save_path)
tokenizer.save_pretrained(final_save_path)
print({final_save_path})

NameError: name 'AutoModelForCausalLM' is not defined

In [3]:
model = AutoModelForCausalLM.from_pretrained(final_save_path)
tokenizer = AutoTokenizer.from_pretrained(final_save_path)

The tokenizer you are loading from '/content/drive/My Drive/deepseek_finetuned_full' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


In [4]:
from transformers import pipeline

In [5]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [9]:
prompt = "23 đầu công việc trong ngày bao gồm những gì?"

In [10]:
# Generate text using the pipeline
generated_texts = pipe(prompt, max_length=300, num_return_sequences=1)
generated_text = generated_texts[0]['generated_text']

Both `max_new_tokens` (=256) and `max_length`(=300) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [11]:
generated_text

'23 đầu công việc trong ngày bao gồm những gì? 23 intersection of sets in probability. 23 permutations and combinations problems. 23 problems of averages. 23 probability questions for class 8. 23 probability questions for class 8. 23 probability questions for class 8. 23 probability questions for class 8. 23 probability questions for class 8. 23 probability questions for class 8. 23 probability questions for class 8.\n\nWait, I think I have a mistake here. I think I have a mistake in the number of things I wrote. I think I have a mistake in the number of things I wrote. I think I have a mistake in the number of things I wrote. I think I have a mistake in the number of things I wrote. I think I have a mistake in the number of things I wrote. I think I have a mistake in the number of things I wrote.\n\nI think I have a mistake in the number of things I wrote. I think I have a mistake in the number of things I wrote. I think I have a mistake in the number of things I wrote. I think I have

In [12]:
# Vừa training xong
trainer.train()

# Load lại để test
model = AutoModelForCausalLM.from_pretrained(final_save_path)
tokenizer = AutoTokenizer.from_pretrained(final_save_path)

# Test
prompt = "Question: Chủ đề lãnh đạo năm 2025 là gì?"
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs)
print(tokenizer.decode(outputs[0]))

NameError: name 'trainer' is not defined